In [1]:
import csv
from datetime import datetime
from pathlib import Path
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
from prettytable import PrettyTable

from utils import *

In [2]:
print(datetime.today().strftime("%Y-%m-%d %H:%M:%S"))
today = datetime.today().strftime("%Y%m%d%H%M%S")

2023-06-13 08:13:54


In [3]:
test_data = [['', 'Train', 'Valid'],
             ['Data size', 10000, 2000]]

x = PrettyTable()
x.field_names = ['', 'Train', 'Valid']
x.add_row(['Data size', 10000, 2000])
print(x)

+-----------+-------+-------+
|           | Train | Valid |
+-----------+-------+-------+
| Data size | 10000 |  2000 |
+-----------+-------+-------+


## Create docker container

```bash
docker run -ti --runtime=runc --name immcantation_cm --workdir /data -v /data/personal/jiewen.du/CM_new:/data immcantation/suite:4.3.0 /bin/bash
```

In [4]:
ROOT = Path('/data/personal/jiewen.du/CM_new')
SCRIPT = ROOT / 'script'
DATA_HOME = ROOT / 'data'

DATA_DIR = DATA_HOME / 'ymc_copy'
CELL_PASS_DIR = DATA_HOME / 'cell_pass'
FMT7_DIR = DATA_HOME / 'fmt7'
FMT19_DIR = DATA_HOME / 'fmt19'

WORK_DIR = Path('/data')
CONTAINER = 'immcantation_cm'

In [8]:
CURRENT_DATASET = ['7_new']

In [6]:
statics_header = ['dataset', 'All cells', 'Non-B cells', 'No-Germlines', 'B cells', 
                  'B cells train', 'B cells train clone_pass', 'unique NT train clone_pass', 
                  'B cells train clone_pass_germ_pass', 'unique NT train clone_pass_germ_pass', 
                  'B cells valid', 'B cells valid clone_pass', 'unique NT valid clone_pass', 
                  'B cells valid clone_pass_germ_pass', 'unique NT valid clone_pass_germ_pass'
                 ]

statistic = {}

## Cell-pass
过滤掉非B cells, CDR3为空，非productive BCR的细胞数据

备注：
1. CM0~CM8: CM*bcr.samples.IGHV.igblast.tsv均为fmt19格式
2. CM9, CM10: CM*bcr.samples.IGHV.igblast.tsv均为fmt7格式  

Source: CM00   | Cells: 56762  | Non-Bcells: 2099   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 54663  
Source: CM1    | Cells: 32201  | Non-Bcells: 4365   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 27836  
Source: CM2    | Cells: 32651  | Non-Bcells: 6202   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 26449  
Source: CM3    | Cells: 53432  | Non-Bcells: 9572   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 43860  
Source: CM4    | Cells: 90319  | Non-Bcells: 2402   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 87917  
Source: CM5    | Cells: 13781  | Non-Bcells: 266    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 13515  
Source: CM5_2  | Cells: 144084 | Non-Bcells: 781    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 143303  
Source: CM6    | Cells: 53426  | Non-Bcells: 1286   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 52140  
Source: CM7    | Cells: 30031  | Non-Bcells: 967    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 29064  
Source: CM8    | Cells: 66527  | Non-Bcells: 1694   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 64833  
Source: CM9    | Cells: 103685 | Non-Bcells: 317    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 103368  
Source: CM10   | Cells: 48743  | Non-Bcells: 340    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 48403    
Source: CM11   | Cells: 204154 | Non-Bcells: 723    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 203431  
Source: CM12   | Cells: 138900 | Non-Bcells: 726    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 138174  
Source: CM13   | Cells: 153536 | Non-Bcells: 788    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 152748  
Source: CM15   | Cells: 152141 | Non-Bcells: 578    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 151563  
Source: CM20   | Cells: 156568 | Non-Bcells: 278    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 156290  
Source: CM21   | Cells: 130004 | Non-Bcells: 286    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 129718   
Source: CM22   | Cells: 137704 | Non-Bcells: 755    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 136949  
Source: CM28   | Cells: 73189  | Non-Bcells: 142    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 73047   
Source: CM29   | Cells: 152204 | Non-Bcells: 197    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 152007  
Source: CM32   | Cells: 135856 | Non-Bcells: 575    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 135281  
Source: CM36   | Cells: 200225 | Non-Bcells: 841    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 199384  

In [66]:
txt ='''Source: CM00   | Cells: 56762  | Non-Bcells: 2099   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 54663  
Source: CM1    | Cells: 32201  | Non-Bcells: 4365   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 27836  
Source: CM2    | Cells: 32651  | Non-Bcells: 6202   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 26449  
Source: CM3    | Cells: 53432  | Non-Bcells: 9572   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 43860  
Source: CM4    | Cells: 90319  | Non-Bcells: 2402   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 87917  
Source: CM5    | Cells: 13781  | Non-Bcells: 266    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 13515  
Source: CM6    | Cells: 53426  | Non-Bcells: 1286   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 52140  
Source: CM7    | Cells: 30031  | Non-Bcells: 967    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 29064  
Source: CM8    | Cells: 66527  | Non-Bcells: 1694   | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 64833  
Source: CM9    | Cells: 103685 | Non-Bcells: 317    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 103368  
Source: CM10   | Cells: 48743  | Non-Bcells: 340    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 48403  '''

txt2 = '''Source: CM00 | Train clone: 21440/54663  | Train germ: 21440/54663  | Valid germ: 2268/3635   | Valid germ: 2268/3635
Source: CM1 | Train clone: 15618/24200  | Train germ: 15618/24200  | Valid germ: 2268/3635   | Valid germ: 2268/3635  
Source: CM2 | Train clone: 17525/23363  | Train germ: 17525/23363  | Valid germ: 2404/3078   | Valid germ: 2404/3078  
Source: CM3 | Train clone: 24016/43190  | Train germ: 24016/43190  | Valid germ: 624/669    | Valid germ: 624/669   
Source: CM4 | Train clone: 46610/86160  | Train germ: 46610/86160  | Valid germ: 1134/1754   | Valid germ: 1134/1754  
Source: CM5 | Train clone: 6167/9736   | Train germ: 6167/9736   | Valid germ: 1099/3778   | Valid germ: 1099/3778  
Source: CM6 | Train clone: 22033/51771  | Train germ: 22033/51771  | Valid germ: 224/366    | Valid germ: 224/366   
Source: CM7 | Train clone: 17321/29044  | Train germ: 17321/29044  | Valid germ: 19/19     | Valid germ: 19/19    
Source: CM8 | Train clone: 27507/64371  | Train germ: 27507/64371  | Valid germ: 276/460    | Valid germ: 276/460   
Source: CM9 | Train clone: 61346/102588 | Train germ: 61346/102588 | Valid germ: 564/662    | Valid germ: 564/662   
Source: CM10 | Train clone: 28987/47865  | Train germ: 28987/47865  | Valid germ: 331/371    | Valid germ: 331/371   '''

data = []
for row in txt2.split('\n'):
    sub_data = []
    for item in row.split(' | '):
        sub_data.append(item.split(':')[-1].strip())
    data.append(sub_data)
data

[['CM00', '21440/54663', '21440/54663', '2268/3635', '2268/3635'],
 ['CM1', '15618/24200', '15618/24200', '2268/3635', '2268/3635'],
 ['CM2', '17525/23363', '17525/23363', '2404/3078', '2404/3078'],
 ['CM3', '24016/43190', '24016/43190', '624/669', '624/669'],
 ['CM4', '46610/86160', '46610/86160', '1134/1754', '1134/1754'],
 ['CM5', '6167/9736', '6167/9736', '1099/3778', '1099/3778'],
 ['CM6', '22033/51771', '22033/51771', '224/366', '224/366'],
 ['CM7', '17321/29044', '17321/29044', '19/19', '19/19'],
 ['CM8', '27507/64371', '27507/64371', '276/460', '276/460'],
 ['CM9', '61346/102588', '61346/102588', '564/662', '564/662'],
 ['CM10', '28987/47865', '28987/47865', '331/371', '331/371']]

In [67]:
with open('table.csv', 'w') as f:
    f_csv = csv.writer(f)
    f_csv.writerows(data)

In [35]:
dataset = 'CM3'
df0 = pd.read_csv(DATA_DIR/f'{dataset}.TSM_VS_VMM_all.bcr.samples.IGHV.igblast.tsv', sep='\t')
df = df0.loc[df0['celltype'].isin(BTYPE.keys()), :].copy(True)
df1 = df0.loc[~df0['celltype'].isin(BTYPE.keys()), :].copy(True)

In [36]:
df1.head(3)

,stim,sequence_id,sequence,rev_comp,productive,v_call,d_call,j_call,sequence_alignment,germline_alignment,...,c_call,consensus_count,umi_count,v_call_10x,d_call_10x,j_call_10x,junction_10x,junction_10x_aa,barcode,celltype
102,CM003-8-1,CAGAATCCAAGTCTAC-1.CM003-8-1,TTTTTCTTATATGGGGAGCTCTCAGAGAGGTGCCTTAGCCCTGGAT...,False,True,IGHV3-48*03,NaN,IGHJ4*02,......CAACTGGTGGAGTCTGGGGGA...GGCTTGGTACAGCCTG...,GAGGTGCAGCTGGTGGAGTCTGGGGGA...GGCTTGGTACAGCCTG...,...,mus-IGHG2B,120223,11424,hsa-IGHV3-48,mus-IGHD3-1,hsa-IGHJ4,TGTGCGAGAGGCGCGGACTACTGG,CARGADYW,CAGAATCCAAGTCTAC-1.CM003-8-1,Monocytes
137,CM003-3-2,AGAGTGGTCCAAATGC-1.CM003-3-2,AATGCGGCACCCTCTTTTCTTATATGGGGGACTCCTGTGCCCCACC...,False,True,IGHV2-26*01,IGHD6-19*01,IGHJ1*01,CAGGTCACCTTGAAGGAGTCTGGTCCT...GTGCTGGTGAAACCCA...,CAGGTCACCTTGAAGGAGTCTGGTCCT...GTGCTGGTGAAACCCA...,...,mus-IGHG1,110962,10858,hsa-IGHV2-26,hsa-IGHD6-19,hsa-IGHJ1,TGTGCACGGTTAGCAGTGGCTGGTACGGCTGAATACTTCCAGCACTGG,CARLAVAGTAEYFQHW,AGAGTGGTCCAAATGC-1.CM003-3-2,Monocytes
183,CM003-8-1,CATGCCTCACTTACGA-1.CM003-8-1,AGGTGTATTGGTTTCTTATATGGGGAGTGCTTTCTGAGAGTCATGG...,False,True,IGHV4-34*01,IGHD3-10*01,IGHJ6*02,CAGGTGCAGCTACAGCAGTGGGGCGCA...GGACTGTTGAAGCCTT...,CAGGTGCAGCTACAGCAGTGGGGCGCA...GGACTGTTGAAGCCTT...,...,mus-IGHG2C,126579,10357,hsa-IGHV4-34,hsa-IGHD3-10,hsa-IGHJ6,TGTGCGAGTGGATCACTATGGTTCGGAGAATTAGCCGTCTACTACT...,CASGSLWFGELAVYYYGMDVW,CATGCCTCACTTACGA-1.CM003-8-1,Monocytes


In [9]:
for dataset in [f'CM{i}' for i in CURRENT_DATASET]:
    df0 = pd.read_csv(DATA_DIR/f'{dataset}.TSM_VS_VMM_all.bcr.samples.IGHV.igblast.tsv', sep='\t')

    df = df0.loc[df0['celltype'].isin(BTYPE.keys()), :].copy(True)
    print('Source: {:<6} | Cells: {:<6} | Non-Bcells: {:<6} '.format(dataset, df0.shape[0], df0.shape[0]-df.shape[0]), end='')
    
    df.reset_index(drop=True, inplace=True)
    df['mice_index'] = df['sequence_id'].apply(lambda x: x.split('.')[-1])

    droped_idx1 = df.loc[(df['cdr3']=='')|(df['cdr3']=='None')|(df['cdr3']=='NA')|(df['cdr3']==None)|(df['cdr3'].isna()), :].index
    df.drop(index=droped_idx1, inplace=True)
    df.reset_index(drop=True, inplace=True)

    droped_idx2 = df.loc[df['productive']==False, :].index
    df.drop(index=droped_idx2, inplace=True)

    # 2: fix wrong dtype
    df = fix_wrong_dtype(df)
    df = df.where(pd.notna(df), None)

    df.to_csv(CELL_PASS_DIR/f'{dataset}.fmt7', sep='\t', index=None, na_rep='')

    with open(CELL_PASS_DIR/f'{dataset}.fasta', 'w') as f:
        for row in df.loc[:, ['sequence_id', 'sequence']].itertuples(False):
            f.write(f'>{row[0]}\n')
            f.write(f'{row[1]}\n')
    
    print('| Null CDR3: {:<6} | Non-productives: {:<6} | Cell-pass: {:<6}'.format(len(droped_idx1), len(droped_idx1), df.shape[0]))
    statistic[dataset] = [dataset, df0.shape[0], df0.shape[0]-df.shape[0]]

Source: CM7_new | Cells: 116683 | Non-Bcells: 956    | Null CDR3: 0      | Non-productives: 0      | Cell-pass: 115727


In [10]:
statistic

{'CM7_new': ['CM7_new', 116683, 956]}

In [5]:
for ifile in DATA_DIR.rglob('*bcr.samples.IGHV.igblast.tsv'):
    source = str(ifile.stem).split('_')[0]
    has_gap = True
    
    df0 = pd.read_csv(ifile, sep='\t')
    
    # 1: cell-pass, cdr3-pass, is_productive-pass:
    df = df0.loc[df0['celltype'].isin(BTYPE.keys()), :].copy(True)
    df.reset_index(drop=True, inplace=True)
    df['mice_index'] = df['sequence_id'].apply(lambda x: x.split('.')[-1])
    
    droped_idx = df.loc[(df['cdr3']=='')|(df['cdr3']=='None')|(df['cdr3']=='NA')|(df['cdr3']==None)|(df['cdr3'].isna()), :].index
    df.drop(index=droped_idx, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    droped_idx = df.loc[df['productive']==False, :].index
    df.drop(index=droped_idx, inplace=True)
    
    # 2: fix wrong dtype
    df = fix_wrong_dtype(df)
    
    df = df.where(pd.notna(df), None)
    
    # 3: split train:valid by mice_index
    if source in ['CM0', 'CM00']:
        with open(CELL_PASS_DIR/f'{source}_train.fasta', 'w') as f:
            for row in df.loc[:, ['sequence_id', 'sequence']].itertuples(False):
                f.write(f'>{row[0]}\n')
                f.write(f'{row[1]}\n')
        
        df.to_csv(CELL_PASS_DIR/f'{source}_train_yao.fmt19', sep='\t', index=None, na_rep='')
        print('Source: {:<6} | Cells: {:<6} | Cell-pass: {:<6}'.format(source, df0.shape[0], df.shape[0]))
        continue
    
    print('Source: {:<6} | Cells: {:<6d} | Cell-pass: {:<6d} | Valid cells: {:<6d} | Train cells: {:<6d}'.format(source, df0.shape[0], df.shape[0], df_v.shape[0], df_t.shape[0]))
    
    # 4: save data as fasta format(>sequence_id, sequence)
#     with open(DATA_DIR2/f'{source}_train.fasta', 'w') as f:
#         for row in df_t.loc[:, ['sequence_id', 'sequence']].itertuples(False):
#             f.write(f'>{row[0]}\n')
#             f.write(f'{row[1]}\n')
    
#     with open(DATA_DIR2/f'{source}_valid.fasta', 'w') as f:
#         for row in df_v.loc[:, ['sequence_id', 'sequence']].itertuples(False):
#             f.write(f'>{row[0]}\n')
#             f.write(f'{row[1]}\n')
    
    with open(DATA_DIR2/f'{source}_all.fasta', 'w') as f:
        for row in df.loc[:, ['sequence_id', 'sequence']].itertuples(False):
            f.write(f'>{row[0]}\n')
            f.write(f'{row[1]}\n')
    
    # 5: save as tsv: fmt7 or fmt19
    if source in ['CM009', 'CM010']:
        df.to_csv(CELL_PASS_DIR/f'{source}_all_yao.fmt7', sep='\t', index=None, na_rep='')
    else:
        df.to_csv(CELL_PASS_DIR/f'{source}_all_yao.fmt19', sep='\t', index=None, na_rep='')

Source: CM010  | Cells: 48743  | Cell-pass: 48403  | Valid cells: 371    | Train cells: 48032 
Source: CM009  | Cells: 92743  | Cell-pass: 92441  | Valid cells: 663    | Train cells: 91778 
Source: CM0    | Cells: 1906   | Cell-pass: 1770  
Source: CM005  | Cells: 2005   | Cell-pass: 1963   | Valid cells: 160    | Train cells: 1803  
Source: CM006  | Cells: 13157  | Cell-pass: 12783  | Valid cells: 126    | Train cells: 12657 
Source: CM004  | Cells: 28339  | Cell-pass: 27679  | Valid cells: 880    | Train cells: 26799 
Source: CM002  | Cells: 13310  | Cell-pass: 11992  | Valid cells: 2184   | Train cells: 9808  
Source: CM003  | Cells: 20614  | Cell-pass: 18869  | Valid cells: 1129   | Train cells: 17740 
Source: CM001  | Cells: 16908  | Cell-pass: 15187  | Valid cells: 1768   | Train cells: 13419 
Source: CM00   | Cells: 18576  | Cell-pass: 17967 
Source: CM008  | Cells: 14761  | Cell-pass: 14561  | Valid cells: 94     | Train cells: 14467 


## Run igblastn(v1.17.0)
进行igblast分析，对BCR序列进行注释

In [11]:
fasta = CELL_PASS_DIR/'CM7_new.fasta'
run_igblast(fasta, fmt=19, threds=50)

# fasta = CELL_PASS_DIR/'CM8_new.fasta'
# run_igblast(fasta, fmt=19, threds=50)

# fasta = CELL_PASS_DIR/'CM40.fasta'
# run_igblast(fasta, fmt=19, threds=50)

Submitted batch job 2


## Make DB
将igblast分析文件转为tab分割的文本文件<br>
注：只能转换‘-outfmt “7 std qseq sseq btop”’的igblast结果文件

In [5]:
db = WORK_DIR / 'data/fmt7/fmt7/CM005_valid.fmt7'
fasta = WORK_DIR / 'data/cell_pass/CM005_valid.fasta'
make_db(db, fasta, CONTAINER, True)

Submitted batch job 44195


In [7]:
for i in range(1, 11):
    if i in [3, 7, 10]:
        continue
    db = WORK_DIR / f'data/fmt7/CM00{i}_all.fmt7'
    fasta = WORK_DIR / f'data/cell_pass/CM00{i}_all.fasta'
    make_db(db, fasta, CONTAINER, True)

Submitted batch job 33828
Submitted batch job 33829
Submitted batch job 33830
Submitted batch job 33831
Submitted batch job 33832
Submitted batch job 33833
Submitted batch job 33834


## Check db
有些比对错误，导致germline_alignment为空  
如果没有比对错误,则不输出*fixed.tsv文件

Source: CM00       | Null germline_alignment: 0      | d_start > d_end: 0      | Passed-cells: 54663  
Source: CM1        | Null germline_alignment: 1      | d_start > d_end: 1      | Passed-cells: 27835  
Source: CM2        | Null germline_alignment: 8      | d_start > d_end: 8      | Passed-cells: 26441  
Source: CM3        | Null germline_alignment: 1      | d_start > d_end: 1      | Passed-cells: 43859  
Source: CM4        | Null germline_alignment: 3      | d_start > d_end: 3      | Passed-cells: 87914  
Source: CM5        | Null germline_alignment: 1      | d_start > d_end: 1      | Passed-cells: 13514  
Source: CM5_2      | Null germline_alignment: 14     | d_start > d_end: 14     | Passed-cells: 143289   
Source: CM6        | Null germline_alignment: 3      | d_start > d_end: 3      | Passed-cells: 52137  
Source: CM7        | Null germline_alignment: 1      | d_start > d_end: 1      | Passed-cells: 29063  
Source: CM8        | Null germline_alignment: 2      | d_start > d_end: 2      | Passed-cells: 64831  
Source: CM9        | Null germline_alignment: 118    | d_start > d_end: 118    | Passed-cells: 103250  
Source: CM10       | Null germline_alignment: 167    | d_start > d_end: 167    | Passed-cells: 48236  
Source: CM11       | Null germline_alignment: 162    | d_start > d_end: 162    | Passed-cells: 203269  
Source: CM12       | Null germline_alignment: 0      | d_start > d_end: 0      | Passed-cells: 138174  
Source: CM13       | Null germline_alignment: 29     | d_start > d_end: 29     | Passed-cells: 152719  
Source: CM15       | Null germline_alignment: 1      | d_start > d_end: 1      | Passed-cells: 151562  
Source: CM20       | Null germline_alignment: 0      | d_start > d_end: 0      | Passed-cells: 156290  
Source: CM21       | Null germline_alignment: 9      | d_start > d_end: 9      | Passed-cells: 129709  
Source: CM22       | Null germline_alignment: 12     | d_start > d_end: 12     | Passed-cells: 136937  
Source: CM28       | Null germline_alignment: 3      | d_start > d_end: 3      | Passed-cells: 73044   
Source: CM29       | Null germline_alignment: 72     | d_start > d_end: 72     | Passed-cells: 151935  

In [12]:
for dataset in [f'CM{i}' for i in CURRENT_DATASET]:
    ifile = CELL_PASS_DIR/f'{dataset}.fmt7'
    df = pd.read_csv(ifile, sep='\t')
    df = fix_wrong_dtype(df)
    
    droped_idx1 = df.loc[df['germline_alignment'].isna(),:].index
    droped_idx2 = df.loc[df['d_germline_start']>=df['d_germline_end'],:].index
    print('Source: {:<10} | Null germline_alignment: {:<6} | d_start > d_end: {:<6} | Passed-cells: {:<6}'.format(dataset, len(droped_idx1), len(droped_idx2), df.shape[0]-len(droped_idx1)))
    statistic[dataset].extend([len(droped_idx1), df.shape[0]-len(droped_idx1)])
    
    if len(droped_idx1)!=0 or len(droped_idx2)!=0:
        df.drop(index=droped_idx1,inplace=True)
        df.to_csv(CELL_PASS_DIR/f'{dataset}_fixed.tsv', sep='\t', index=None, na_rep='')

print(statistic)

Source: CM7_new    | Null germline_alignment: 3      | d_start > d_end: 3      | Passed-cells: 115724
{'CM7_new': ['CM7_new', 116683, 956, 3, 115724]}


## split train, valid

Source: CM1    | DB-pass: 27835  | Train cells: 24200  | Valid cells: 3635  
Source: CM2    | DB-pass: 26441  | Train cells: 23363  | Valid cells: 3078  
Source: CM3    | DB-pass: 43859  | Train cells: 43190  | Valid cells: 669   
Source: CM4    | DB-pass: 87914  | Train cells: 86160  | Valid cells: 1754  
Source: CM5    | DB-pass: 13514  | Train cells: 9736   | Valid cells: 3778   
Source: CM5_2  | DB-pass: 143289 | Train cells: 136423 | Valid cells: 6866  
Source: CM6    | DB-pass: 52137  | Train cells: 51771  | Valid cells: 366   
Source: CM7    | DB-pass: 29063  | Train cells: 29044  | Valid cells: 19  
Source: CM8    | DB-pass: 64831  | Train cells: 64371  | Valid cells: 460    
Source: CM9    | DB-pass: 103250 | Train cells: 102588 | Valid cells: 662   
Source: CM10   | DB-pass: 48236  | Train cells: 47865  | Valid cells: 371  
Source: CM11   | DB-pass: 203269 | Train cells: 189877 | Valid cells: 13392   
Source: CM20   | DB-pass: 156290 | Train cells: 155597 | Valid cells: 693   
Source: CM22   | DB-pass: 136937 | Train cells: 123434 | Valid cells: 13503  
Source: CM28   | DB-pass: 73044  | Train cells: 71366  | Valid cells: 1678   
Source: CM29   | DB-pass: 151935 | Train cells: 145505 | Valid cells: 6430   

In [13]:
dataset = 'CM7_new'
ifile = DATA_HOME/f'ymc_copy/{dataset}.TSM_VS_VMM_all.bcr.samples.IGHV.igblast.tsv'
df_temp = pd.read_csv(ifile, sep='\t')
df_temp.stim.value_counts()

CM-007-2-1-1    18168
CM-007-2-1-6    17137
CM-007-2-1-2    17069
CM-007-2-1-5    16324
CM-007-2-1-4    15306
CM-007-2-1-7    15244
CM-007-2-1-3    13355
CM-007-2-2-1     4080
Name: stim, dtype: int64

In [14]:
VALID_MICE.update({
    'CM7_new': [['CM-007-2-2-1']]
    # 'CM39': [['CM039-2-1'], ['CM039-3-1', 'CM039-3-2', 'CM039-3-3']]
})

for dataset in [f'CM{i}' for i in CURRENT_DATASET]:
    ifile = CELL_PASS_DIR/f'{dataset}_fixed.tsv'
    if not ifile.exists():
        ifile = CELL_PASS_DIR/f'{dataset}.fmt7'
    
    df = pd.read_csv(ifile, sep='\t')
    # df['mice_index'] = df['sequence_id'].apply(lambda x: x.split('.')[1])
    df = fix_wrong_dtype(df)
    
    df_tp = df.loc[df['mice_index'].isin(VALID_MICE[dataset][0]), :]
    if len(VALID_MICE[dataset])>1:
        df_tn = df.loc[df['mice_index'].isin(VALID_MICE[dataset][1]), :]
        df_t = df.loc[(~df.index.isin(df_tp.index))&(~df.index.isin(df_tn.index)), :]
        df_tn.to_csv(CELL_PASS_DIR/f'{dataset}_valid_tn.tsv', sep='\t', index=None, na_rep='')
        df_tp.to_csv(CELL_PASS_DIR/f'{dataset}_valid_tp.tsv', sep='\t', index=None, na_rep='')
    else:
        df_t = df.loc[~df.index.isin(df_tp.index), :]
        df_tp.to_csv(CELL_PASS_DIR/f'{dataset}_valid.tsv', sep='\t', index=None, na_rep='')
    
    df_t.to_csv(CELL_PASS_DIR/f'{dataset}_train.tsv', sep='\t', index=None, na_rep='')
    print('Source: {:<6} | DB-pass: {:<6} | Train cells: {:<6d} | Valid cells: {:<6d}'.format(dataset, df.shape[0], df_t.shape[0], df_tp.shape[0]))

Source: CM7_new | DB-pass: 115724 | Train cells: 111713 | Valid cells: 4011  


## Find optimal distance threshold

In [15]:
sh_file = SCRIPT / 'find_threshold.sh'
sh_out = SCRIPT / f'log/slurm_find_threshold.out'

with open(sh_file, 'w') as f:
    f.write('#!/bin/bash\n\n')

    f.write('#SBATCH -n 1 # 指定核心数量\n')
    f.write('#SBATCH -N 1 # 指定node的数量\n')
    f.write('#SBATCH -t 0-02:00 # 运行总时间，天数-小时数-分钟, D-HH:MM\n')
    f.write('#SBATCH --mem=5G # 所有核心可以使用的内存池大小, MB为单位\n')
    f.write(f'#SBATCH -o {sh_out} # 把输出结果STDOUT保存在哪一个文件\n')
    f.write('#SBATCH --nodelist=ab03\n\n')

    f.write("docker start {}\n".format(CONTAINER))
    now = datetime.now().strftime('%Y%m%d%H%M%S')
    stdout = WORK_DIR / f'data/log/findThreshold_{now}.out'
    code = f'''docker exec {CONTAINER} /bin/bash -c "(time Rscript findThreshold.r) > {stdout} 2>&1"\n'''
    f.write('{}\n'.format(code))

os.system(f'sbatch {sh_file}')

Submitted batch job 3


0

## Define clones

In [16]:
df = pd.read_csv(DATA_HOME/'threshold_CM_20220809.csv')

for row in df.iloc[76:, :].itertuples(False):
    if pd.isna(row[1]):
        print(f'No threshold of {row[0]}')
        continue
    print(row[0])
    define_clones(Path(row[0]), row[1], CONTAINER, True)

/data/data/cell_pass/CM7_new_train.tsv
Submitted batch job 4
/data/data/cell_pass/CM7_new_valid.tsv
Submitted batch job 5


## Create germline

In [17]:
for dataset in [f'CM{i}' for i in CURRENT_DATASET]:
    dbs = (DATA_HOME/'clones').glob(f'{dataset}*')
    for db in dbs:
        db_in_docker = WORK_DIR/'data/clones' / db.name
        create_germline(db_in_docker, CONTAINER, True)

Submitted batch job 6
Submitted batch job 7


## Merge *_aa columns in fmt19 with *clone_pass_germ-pass.tsv

In [18]:
for dataset in [f'CM{i}' for i in CURRENT_DATASET]:
    # if dataset=='CM00':
    #     germ_f = HOME/f'clones/{dataset}_train-db_pass_clone-pass_germ-pass.tsv'
    # else:
    #     germ_f = HOME/f'clones/{dataset}_valid-db_pass_clone-pass_germ-pass.tsv'
    
    db = DATA_HOME/f'fmt19/{dataset}.fmt19'
    df_fmt19 = pd.read_csv(db, sep='\t')
    cols = [i for i in df_fmt19.columns if i.find('_aa')>=0]
    cols.remove('junction_aa')
    cols.append('sequence_id')
    
    germ_files = (DATA_HOME/'clones').glob(f'{dataset}*germ-pass*')
    for germ_f in germ_files:
        df = pd.read_csv(germ_f, sep='\t')
        df2 = pd.merge(df, df_fmt19.loc[:, cols], on='sequence_id', how='left')
        df2 = fix_wrong_dtype(df2)
        df2.to_csv(germ_f.parent/f'{germ_f.stem}_merged.tsv', sep='\t', index=None, na_rep='')
        print('Source: {:<6} | File: {:<} | Source shape: {:<6d} | Merged shape: {:<6d}'.format(dataset, germ_f.stem, df.shape[0], df2.shape[0]))

Source: CM7_new | File: CM7_new_valid-clone_pass_germ-pass | Source shape: 4011   | Merged shape: 4011  
Source: CM7_new | File: CM7_new_train-clone_pass_germ-pass | Source shape: 111713 | Merged shape: 111713


In [19]:
for key, val in statistic.items():
    statistic[key] = val[:5]
statistic

{'CM7_new': ['CM7_new', 116683, 956, 3, 115724]}

# 统计数据

In [20]:
for dataset in [f'CM{i}' for i in CURRENT_DATASET]:
    clone_f_train = DATA_HOME/f'clones/{dataset}_train-clone_pass.tsv'
    germ_f_train = DATA_HOME/f'clones/{dataset}_train-clone_pass_germ-pass.tsv'
    df_clone_train = pd.read_csv(clone_f_train, sep='\t')
    df_germ_train = pd.read_csv(germ_f_train, sep='\t')
    
    clone_f_valid = DATA_HOME/f'clones/{dataset}_valid-clone_pass.tsv'
    germ_f_valid = DATA_HOME/f'clones/{dataset}_valid-clone_pass_germ-pass.tsv'
    if clone_f_valid.exists():
        df_clone_valid = pd.read_csv(clone_f_valid, sep='\t')
        df_germ_valid = pd.read_csv(germ_f_valid, sep='\t')
        
        statistic[dataset].extend([df_clone_train.shape[0],
                                    df_clone_train.shape[0],
                                    len(set(df_clone_train.sequence_alignment)),
                                   df_germ_train.shape[0],
                                    len(set(df_germ_train.sequence_alignment)),
                                   df_clone_valid.shape[0],
                                    df_clone_valid.shape[0],
                                    len(set(df_clone_valid.sequence_alignment)),
                                   df_germ_valid.shape[0],
                                    len(set(df_germ_valid.sequence_alignment))])
        
    else:
        clone_f_valid_tp = DATA_HOME/f'clones/{dataset}_valid_tp-clone_pass.tsv'
        germ_f_valid_tp = DATA_HOME/f'clones/{dataset}_valid_tp-clone_pass_germ-pass.tsv'
        clone_f_valid_tn = DATA_HOME/f'clones/{dataset}_valid_tn-clone_pass.tsv'
        germ_f_valid_tn = DATA_HOME/f'clones/{dataset}_valid_tn-clone_pass_germ-pass.tsv'
        
        df_clone_valid_tp = pd.read_csv(clone_f_valid_tp, sep='\t')
        df_germ_valid_tp = pd.read_csv(germ_f_valid_tp, sep='\t')
        df_clone_valid_tn = pd.read_csv(clone_f_valid_tn, sep='\t')
        df_germ_valid_tn = pd.read_csv(germ_f_valid_tn, sep='\t')
        
        statistic[dataset].extend([df_clone_train.shape[0],
                                    df_clone_train.shape[0],
                                    len(set(df_clone_train.sequence_alignment)),
                                   df_germ_train.shape[0],
                                    len(set(df_germ_train.sequence_alignment)),
                                   df_clone_valid_tp.shape[0],
                                   df_clone_valid_tp.shape[0],
                                    len(set(df_clone_valid_tp.sequence_alignment)),
                                   df_germ_valid_tp.shape[0],
                                    len(set(df_germ_valid_tp.sequence_alignment)),
                                   df_clone_valid_tn.shape[0],
                                    df_clone_valid_tn.shape[0],
                                    len(set(df_clone_valid_tn.sequence_alignment)),
                                   df_germ_valid_tn.shape[0],
                                    len(set(df_germ_valid_tn.sequence_alignment))])
        
print(statistic)

{'CM7_new': ['CM7_new', 116683, 956, 3, 115724, 111713, 111713, 45750, 111713, 45750, 4011, 4011, 2055, 4011, 2055]}


In [21]:
with open(DATA_HOME/'statistic.csv', 'a+') as f:
    f_csv = csv.writer(f)
    # f_csv.writerow(statics_header)
    f_csv.writerows(statistic.values())

# Calculate mutation

In [ ]:
!docker exec $CONTAINER /bin/bash -c "Rscript mutation_statistic.r"

## Statistic

Source: CM00 | Train clone: 21440/54663  | Train germ: 21440/54663   
Source: CM1 | Train clone: 15618/24200  | Train germ: 15618/24200  | Valid germ: 2268/3635   | Valid germ: 2268/3635   
Source: CM2 | Train clone: 17525/23363  | Train germ: 17525/23363  | Valid germ: 2404/3078   | Valid germ: 2404/3078  
Source: CM3 | Train clone: 24016/43190  | Train germ: 24016/43190  | Valid germ: 624/669    | Valid germ: 624/669   
Source: CM4 | Train clone: 46610/86160  | Train germ: 46610/86160  | Valid germ: 1134/1754   | Valid germ: 1134/1754  
Source: CM5 | Train clone: 6167/9736   | Train germ: 6167/9736   | Valid germ: 1099/3778   | Valid germ: 1099/3778  
Source: CM6 | Train clone: 22033/51771  | Train germ: 22033/51771  | Valid germ: 224/366    | Valid germ: 224/366   
Source: CM7 | Train clone: 17321/29044  | Train germ: 17321/29044  | Valid germ: 19/19     | Valid germ: 19/19    
Source: CM8 | Train clone: 27507/64371  | Train germ: 27507/64371  | Valid germ: 276/460    | Valid germ: 276/460   
Source: CM9 | Train clone: 61346/102588 | Train germ: 61346/102588 | Valid germ: 564/662    | Valid germ: 564/662   
Source: CM10 | Train clone: 28987/47865  | Train germ: 28987/47865  | Valid germ: 331/371    | Valid germ: 331/371   
Source: CM12 | Train clone: 66969/129047 | Train germ: 66969/129047 | Valid germ: 3285/9127   | Valid germ: 3285/9127  
Source: CM13 | Train clone: 79820/146466 | Train germ: 79815/146461 | Valid germ: 1919/6253   | Valid germ: 1919/6253  
Source: CM15 | Train clone: 69019/141947 | Train germ: 69019/141947 | Valid germ: 4611/9615   | Valid germ: 4611/9615  
Source: CM21 | Train clone: 75322/122619 | Train germ: 75322/122619 | Valid germ: 3443/7090   | Valid germ: 3443/7090  

In [22]:
for dataset in [f'CM{i}' for i in ['5_2',11,20,22,28,29]]:
    clone_f_t = DATA_HOME/f'clones/{dataset}_train-clone_pass.tsv'
    clone_f_v = DATA_HOME/f'clones/{dataset}_valid-clone_pass.tsv'
    germ_f_t = DATA_HOME/f'clones/{dataset}_train-clone_pass_germ-pass.tsv'
    germ_f_v = DATA_HOME/f'clones/{dataset}_valid-clone_pass_germ-pass.tsv'

    clone_df_t = pd.read_csv(clone_f_t, sep='\t', usecols=['sequence_id', 'sequence_alignment'])
    germ_df_t = pd.read_csv(germ_f_t, sep='\t', usecols=['sequence_id', 'sequence_alignment'])
    if dataset!='CM0':
        clone_df_v = pd.read_csv(clone_f_v, sep='\t', usecols=['sequence_id', 'sequence_alignment'])
        germ_df_v = pd.read_csv(germ_f_v, sep='\t', usecols=['sequence_id', 'sequence_alignment'])
        print(f'Source: {dataset} | Train clone: {clone_df_t.sequence_alignment.unique().shape[0]}/{clone_df_t.shape[0]:<6d} | Train germ: {germ_df_t.sequence_alignment.unique().shape[0]}/{germ_df_t.shape[0]:<6d} | Valid germ: {clone_df_v.sequence_alignment.unique().shape[0]}/{clone_df_v.shape[0]:<6d} | Valid germ: {germ_df_v.sequence_alignment.unique().shape[0]}/{germ_df_v.shape[0]:<6d}')
    else:                   
        print(f'Source: {dataset} | Train clone: {clone_df_t.sequence_alignment.unique().shape[0]}/{clone_df_t.shape[0]:<6d} | Train germ: {germ_df_t.sequence_alignment.unique().shape[0]}/{germ_df_t.shape[0]:<6d}')

Source: CM5_2 | Train clone: 65359/136423 | Train germ: 65359/136423 | Valid germ: 2842/6866   | Valid germ: 2842/6866  
Source: CM11 | Train clone: 40730/189877 | Train germ: 40730/189877 | Valid germ: 7556/13392  | Valid germ: 7556/13392 
Source: CM20 | Train clone: 85732/155597 | Train germ: 85732/155597 | Valid germ: 549/693    | Valid germ: 549/693   
Source: CM22 | Train clone: 59482/123434 | Train germ: 59482/123434 | Valid germ: 4204/13503  | Valid germ: 4204/13503 
Source: CM28 | Train clone: 41614/71366  | Train germ: 41614/71366  | Valid germ: 966/1678   | Valid germ: 966/1678  
Source: CM29 | Train clone: 99292/145505 | Train germ: 99292/145505 | Valid germ: 3702/6430   | Valid germ: 3702/6430  


## Build tree

In [35]:
db = DATA_HOME/'clones/CM20_valid-clone_pass_germ-pass.tsv'
build_repertoire_tree(db, CONTAINER, 3, 500)

Submitted batch job 66091
Submitted batch job 66092
Submitted batch job 66093
Submitted batch job 66094
Submitted batch job 66095
Submitted batch job 66096
Submitted batch job 66097
Submitted batch job 66098
Submitted batch job 66099
Submitted batch job 66100
Submitted batch job 66101
Submitted batch job 66102
Submitted batch job 66103
Submitted batch job 66104
Submitted batch job 66105
Submitted batch job 66106
Submitted batch job 66107
Submitted batch job 66108
Submitted batch job 66109
Submitted batch job 66110
Submitted batch job 66111
Submitted batch job 66112
Submitted batch job 66113
Submitted batch job 66114
Submitted batch job 66115
Submitted batch job 66116
Submitted batch job 66117
Submitted batch job 66118
Submitted batch job 66119
Submitted batch job 66120
Submitted batch job 66121
Submitted batch job 66122
Submitted batch job 66123
Submitted batch job 66124
Submitted batch job 66125
Submitted batch job 66126
Submitted batch job 66127


In [37]:
db = DATA_HOME/f'clones/CM5_2_valid-clone_pass_germ-pass.tsv'
rebuild_tree(db, CONTAINER, 3, do_run=True)

Rebuild Tree: CM5_2_valid-clone_pass_germ-pass.tsv


In [27]:
for i in range(1,11):
    if i==7:
        continue
    db = HOME/f'clones/CM{i:0=3}_train-clone_pass_germ-pass.tsv'
    rebuild_tree(db, CONTAINER, 3, do_run=False)

Rebuild Tree: CM001_train-clone_pass_germ-pass.tsv
Rebuild Tree: CM002_train-clone_pass_germ-pass.tsv
Rebuild Tree: CM003_train-clone_pass_germ-pass.tsv
Rebuild Tree: CM004_train-clone_pass_germ-pass.tsv
Rebuild Tree: CM005_train-clone_pass_germ-pass.tsv
Rebuild Tree: CM006_train-clone_pass_germ-pass.tsv
Rebuild Tree: CM008_train-clone_pass_germ-pass.tsv
Rebuild Tree: CM009_train-clone_pass_germ-pass.tsv
Rebuild Tree: CM010_train-clone_pass_germ-pass.tsv


In [5]:
germ_file = WORK_DIR/'data/clones/CM002_valid-clone-pass_germ-pass.tsv'

build_lineage_tree(germ_file, CONTAINER, 99, 20, 3, True, True)

Submitted batch job 35073


In [41]:
df30= pd.read_csv(DATA_HOME/'ymc_copy/CM3_all.bcr.samples.IGHV.igblast.tsv', sep='\t')
df31= pd.read_csv(DATA_HOME/'clones/CM3_valid_clone-pass_germ-pass_merged.tsv', sep='\t')

pos_ids = [
    'CGTCAGGAGTCAATAG-1.VMM_1',
    'GTACGTAGTTTACTCT-1.VMM_1',
    'CATTCGCCAACTGCGC-1.VMM_5_9',
    'TTCTCAATCCTGTACC-1.VMM_5_9',
    'AACTCAGGTCCCTTGT-1.VMM_5_9',
    'CTGGTCTCATTAACCG-1.VMM_5_9',
    'GCATGCGTCACAGGCC-1.VMM_5_9',
    'TGGGCGTTCCACGACG-1.VMM_5_9',
    'CAGCCGACAAGGACAC-1.VMM_5_9',
    'TGGGCGTTCCACGACG-1.VMM_5_9',
    'GAAGCAGGTCATCCCT-1.VMM_1',
    'GCTGCGAAGATATGCA-1.VMM_5_9',
    'CCATGTCTCTCGCATC-1.VMM_5_9',
    'GATCGCGCATCCCACT-1.VMM_5_9'
]

neg_ids = [
    'GAGGTGATCTCAACTT-1.VMM_5_9',
    'CGGTTAAAGAGTCTGG-1.VMM_5_9',
    'CTACACCTCGTCCAGG-1.VMM_1',
    'GCATGATCACGAAAGC-1.VMM_5_9',
    'CGTTAGAGTTGCTCCT-1.VMM_5_9',
    'TGAGCCGTCCACGAAT-1.VMM_5_9',
    'GACACGCTCCTGCTTG-1.VMM_5_9'
]

seq_aa = {
'QVQLQESGPGLVKPSGTLSLTCSVSGGSISNTNWWSWVRQPPGKGLEWIGEIYHSGSTKYNPSLKSRVTISVDKSKNQFSLKFYSVTAADTAVYYCARASFDILTGYYGMDVWGQGTTVTVSS':1,
'QVQLQESGPGLVKPSGTLSLICAVSGGSISSTNWWSWVRQAPEKGLEWIGEIYHSGSTRYNPSLKSRVTISVDKSKNQFSLKLTSVTAADTAVYYCARAYYDVLTGYDGLDVWGLGTSVTVSS':0,
'QVQLQESGPGLVKPSGTLSLTCAVSDGSISSTNWWSWVRQSPGKGLEWIGEIYHSGSTKYNPSLKSRVTILVDKSKNQFSLKLNSVTAADTAVYYCARASFDLLTGYEGLDVWGQGTTVTVSS':0.5,
'QVQLQESGPGLVKPSGTLSLTCAVSGGSISSSNWWSWVRQSPGKGLEWIGEIYHSGSTKYNPSLKSRVTISLDKSENQFSLNLYSVTAADSAVYYCARAAFDILTGFYGLDVWGQGTTVTVSS':0,
'QVQLQESGPELVKPSGTLSLTCAVSGGSITSTYWWTWVRQPPGKGLEWIGEIFHSGSTKYNPSLKSRVTFSVDKSKNQFSLKLTSVTDADTAVYYCARASFDILTGYSGLDVWGQGTTVTVSS':0,
'QLQLQESGPGLVKPSETLSLTCTVSGGSINSGDYYWGWIRQPPGKGLEWIGSISYSGSTYSKPSLKSRVTISVDTSKNQFSLKLSSVTAADTAFYHCARQGRFLDWLLYFDYWGQGALVTVSS':1, 
'QLQLQASGPGLVKPSETLSLTCTVSGGSISRSDNYWGWIRQPPGKGLEWIGSISSSGTTYYNPSLKSRVTISVDTSKDQFSLNLTSVTAADTAVYYCVRQGRFLAWLLYFELWDQGTLVTVSS':1,
'HLQLQESGPGLVKPSETLALTCNVSDGSISSGEYYWGWIRQPPGKGLEWIGSVDYGGTTYYNPSLKSQATISVDTSKNQFSLKLSSVTAADTAVYYCVRQGRFLNWLLYFEDWGRGTLVTVSS':1, 
'QLQLQESGPGPVKPSETLSLTCSVSGGSISSGNYYWGWIRQSPGKGLDWIGSIYYGGTTYYNPSLKSRVTISVDTSMNQFSLHLSSVTAADTAMYFCVKQGRFLDWLLYFDYWGQGTLVIVSS':1,
'QLQLQESGPGLVKPSETLSLTCTVSGDSVSSGDYYWAWIRQPPGKGLEWIGTIYYSGTTYYNPSLKSRVTISVTTSKNQFSLKLNSVTATDTAVYYCARQGRFLDWLLYFENWGQGNLVTVSS':1, 
'QVQLQQWGAGLLKPSETLSLTCAVYGGSLSGNYWSWIRQSPGKGLEWIGEINHYGSTNYNPSLKSRVTISLDTPNNQFSLKLSSVTAADTAVYYCAREGYDILTGVERDFHYWGQGILVTVSS':0.5,
'QVQLQQWGAGLLKPSETLSLTCAVYGGSFKNYYWSWIRQSPGKGLEWIGEISHYGNTNYIPSLKSRVTISLDPSTNQFSLKLTSVTAADTAVYYCARQGYDILTGVERHFDSWGQGTLVTVSS':1,
'QVQLQQWGAGLLKPSETLSLTCAIYGGSFSGYWWTWIRQSPGKGLEWIGEISHSGNTNYNPSLKSRVTITLDTSKNQFSLNLSSVAAADTAIYYCAREGYDILTGLERHFDSWGQGILVTVSS':1,
'QVQLQQWGAGLLKPSETLSLTCAVYDGSLSGYYWSWIRQSPGKGLEWIGEINHYGGTNYNPSLKSRVTISRDTSKNQFSLKLSSVTGADTAVYYCARQGYDILTGVERDFDSWGQGTLVTVSS':0,
'QVQLQQWGAGLLKPSETLSLTCAVNGGSFSGYYWTWIRQSPGKGLEWIGEINHSGSITNYSPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYFCARQGYDILTGVERHFDYWGQGTVVTVSS':0,
'QVQLLESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVTFIWYDQNNKYYADSVKGRFTISRDNSKKMLYLQMNSLRVEDTAVYYCARERETTFSHYYGMDVWGRGTTVTVSS':1,
'QVQVQESGPGLVKPSGTLSLTCAVSGGSMRSSNWWSWVRQSPGKGLEWIGEIHYSGNTNYNPSLKSRVTMSVDKSENQFSLKLTSVTAADTAVYYCARVGHDILTGLNWFDPWGQGSPVIVSS':0,
'QVQLQQSGPGLVKPSQTLSVTCVISGDSVSTSSAVWNWIRQSPSRGLEWLGRTYYRSKWYNDYAISVKSRITINPDTSKNQFSLHLTSVTPEDTAVYYCARGGSSSYDYYGFDVWGQGTTVTVSS':1,
'QVQLQQWGAGLLKPSETLSLTCAVSGGSFSGYYWSWIRQSPGKGLEWIGEINHSGSITNYSPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYFCAREDYDILTGVERDFDYWGQGTVVTVSS':0
}

In [39]:
len(pos_ids)+len(neg_ids)

21

In [36]:
def convert_id(barcode):
    nt = barcode.split('.')[0]
    mouse = barcode.split('.')[-1].replace('CM003', 'VMM').replace('-', '_')
    return f'{nt}.{mouse}'

df3['new_id'] = df3['barcode'].apply(convert_id)

In [37]:
df3.loc[df3.new_id.isin(pos_ids), :]

,stim,sequence_id,sequence,rev_comp,productive,v_call,d_call,j_call,sequence_alignment,germline_alignment,...,consensus_count,umi_count,v_call_10x,d_call_10x,j_call_10x,junction_10x,junction_10x_aa,barcode,celltype,new_id
29625,CM003-1,GTACGTAGTTTACTCT-1.CM003-1,TGGGGATGCTTTCTGAGAGTCATGGATCTCATGTGCAAGAAAATGA...,False,True,IGHV4-39*01,"IGHD3/OR15-3a*01,IGHD3/OR15-3b*01",IGHJ4*02,CAACTACAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,CAGCTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,...,7953,14,hsa-IGHV4-39,hsa-IGHD3/OR15-3A,hsa-IGHJ4,TGTGCGAGACAAGGCCGATTTTTGGACTGGTTATTGTACTTTGAGA...,CARQGRFLDWLLYFENW,GTACGTAGTTTACTCT-1.CM003-1,Plasma cells,GTACGTAGTTTACTCT-1.VMM_1
29643,CM003-1,CGTCAGGAGTCAATAG-1.CM003-1,GACCCTGCAGCTCTGGGAGAGGAGCCCAGCACTAGAAGTCGGCGGT...,False,True,IGHV3-33*01,"IGHD4-11*01,IGHD4-17*01,IGHD4-23*01",IGHJ6*02,CAGGTGCAGTTGCTGGAATCTGGGGGA...GGCGTGGTCCAGCCTG...,CAGGTGCAGCTGGTGGAGTCTGGGGGA...GGCGTGGTCCAGCCTG...,...,3504,14,hsa-IGHV3-33,mus-IGHD5-5,hsa-IGHJ6,TGTGCGAGAGAGAGAGAGACTACCTTTTCCCACTACTACGGTATGG...,CARERETTFSHYYGMDVW,CGTCAGGAGTCAATAG-1.CM003-1,Memory B-cells,CGTCAGGAGTCAATAG-1.VMM_1
30822,CM003-1,GAAGCAGGTCATCCCT-1.CM003-1,GATATACTTTCTGAGACTCATGGACCTCCTGCACAAGAACATGAAA...,False,True,IGHV4-4*02,IGHD3-9*01,IGHJ6*02,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,...,4586,12,hsa-IGHV4-4,hsa-IGHD3-9,hsa-IGHJ6,TGTGCGAGAGCGTCTTTCGATCTTTTGACTGGTTATGAGGGATTGG...,CARASFDLLTGYEGLDVW,GAAGCAGGTCATCCCT-1.CM003-1,Class-switched memory B-cells,GAAGCAGGTCATCCCT-1.VMM_1


In [38]:
df3.loc[df3.new_id.isin(neg_ids), :]

,stim,sequence_id,sequence,rev_comp,productive,v_call,d_call,j_call,sequence_alignment,germline_alignment,...,consensus_count,umi_count,v_call_10x,d_call_10x,j_call_10x,junction_10x,junction_10x_aa,barcode,celltype,new_id
3520,CM003-1,CTACACCTCGTCCAGG-1.CM003-1,GGGAGGGTCCAGCTCACATGGGAAATACTTTCTGAGACTCATGGAC...,False,True,IGHV4-4*02,IGHD3-9*01,IGHJ6*02,CAGGTGCAGCTACAGGAGTCGGGTCCA...GGACTGGTGAAGCCTT...,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,...,99642,4099,hsa-IGHV4-4,hsa-IGHD3-9,hsa-IGHJ6,TGTGCGAGAGCTTATTACGATGTTTTGACTGGTTATGACGGTTTGG...,CARAYYDVLTGYDGLDVW,CTACACCTCGTCCAGG-1.CM003-1,Plasma cells,CTACACCTCGTCCAGG-1.VMM_1
12119,CM003-5-9,CGTTAGAGTTGCTCCT-1.CM003-5-9,GACATTTCCTTAAATTCAGGGTCCAGCTCACATGGGAAATACTTTC...,False,True,IGHV4-4*02,IGHD3-9*01,IGHJ6*02,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GAACTGGTGAAGCCTT...,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,...,9223,237,hsa-IGHV4-4,hsa-IGHD3-9,hsa-IGHJ6,TGTGCGAGGGCCTCTTTCGATATTTTGACTGGTTACTCCGGACTGG...,CARASFDILTGYSGLDVW,CGTTAGAGTTGCTCCT-1.CM003-5-9,Plasma cells,CGTTAGAGTTGCTCCT-1.VMM_5_9


In [42]:
df31.loc[df31.sequence_alignment_aa.isin(seq_aa.keys()), :]

,sequence_id,sequence,rev_comp,productive,v_call,d_call,j_call,sequence_alignment,germline_alignment,junction,...,j_sequence_alignment_aa,j_germline_alignment_aa,fwr1_aa,cdr1_aa,fwr2_aa,cdr2_aa,fwr3_aa,fwr4_aa,cdr3_aa,junction_aa_length
8,CACTCCAAGCTACCGC-1.CM003-5-9,GGGGAGGGTCCAGCTCACATGGGAAAGACTTTCTGAGACTCATGGA...,F,T,IGHV4-4*02,IGHD3-9*01,IGHJ5*02,CAGGTGCAGGTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,TGTGCGAGAGTTGGTCACGATATTTTGACTGGCCTCAACTGGTTCG...,...,NWFDPWGQGSPVIVSS,NWFDPWGQGTLVTVSS,QVQVQESGPGLVKPSGTLSLTCAVS,GGSMRSSNW,WSWVRQSPGKGLEWIGE,IHYSGNT,NYNPSLKSRVTMSVDKSENQFSLKLTSVTAADTAVYYC,WGQGSPVIVSS,ARVGHDILTGLNWFDP,18
125,GAAGCAGGTCATCCCT-1.CM003-1,GATATACTTTCTGAGACTCATGGACCTCCTGCACAAGAACATGAAA...,F,T,IGHV4-4*02,IGHD3-9*01,IGHJ6*02,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,TGTGCGAGAGCGTCTTTCGATCTTTTGACTGGTTATGAGGGATTGG...,...,DVWGQGTTVTVSS,DVWGQGTTVTVSS,QVQLQESGPGLVKPSGTLSLTCAVS,DGSISSTNW,WSWVRQSPGKGLEWIGE,IYHSGST,KYNPSLKSRVTILVDKSKNQFSLKLNSVTAADTAVYYC,WGQGTTVTVSS,ARASFDLLTGYEGLDV,18
158,CGTTAGAGTTGCTCCT-1.CM003-5-9,GACATTTCCTTAAATTCAGGGTCCAGCTCACATGGGAAATACTTTC...,F,T,IGHV4-4*02,IGHD3-9*01,IGHJ6*02,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GAACTGGTGAAGCCTT...,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,TGTGCGAGGGCCTCTTTCGATATTTTGACTGGTTACTCCGGACTGG...,...,YSGLDVWGQGTTVTVSS,YYGMDVWGQGTTVTVSS,QVQLQESGPELVKPSGTLSLTCAVS,GGSITSTYW,WTWVRQPPGKGLEWIGE,IFHSGST,KYNPSLKSRVTFSVDKSKNQFSLKLTSVTDADTAVYYC,WGQGTTVTVSS,ARASFDILTGYSGLDV,18
176,CTACACCTCGTCCAGG-1.CM003-1,GGGAGGGTCCAGCTCACATGGGAAATACTTTCTGAGACTCATGGAC...,F,T,IGHV4-4*02,IGHD3-9*01,IGHJ6*02,CAGGTGCAGCTACAGGAGTCGGGTCCA...GGACTGGTGAAGCCTT...,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,TGTGCGAGAGCTTATTACGATGTTTTGACTGGTTATGACGGTTTGG...,...,GLDVWGLGTSVTVSS,GMDVWGQGTTVTVSS,QVQLQESGPGLVKPSGTLSLICAVS,GGSISSTNW,WSWVRQAPEKGLEWIGE,IYHSGST,RYNPSLKSRVTISVDKSKNQFSLKLTSVTAADTAVYYC,WGLGTSVTVSS,ARAYYDVLTGYDGLDV,18
198,GTACGTAGTTTACTCT-1.CM003-1,TGGGGATGCTTTCTGAGAGTCATGGATCTCATGTGCAAGAAAATGA...,F,T,IGHV4-39*01,"IGHD3/OR15-3a*01,IGHD3/OR15-3b*01",IGHJ4*02,CAACTACAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,CAGCTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTT...,TGTGCGAGACAAGGCCGATTTTTGGACTGGTTATTGTACTTTGAGA...,...,YFENWGQGNLVTVSS,YFDYWGQGTLVTVSS,QLQLQESGPGLVKPSETLSLTCTVS,GDSVSSGDYY,WAWIRQPPGKGLEWIGT,IYYSGTT,YYNPSLKSRVTISVTTSKNQFSLKLNSVTATDTAVYYC,WGQGNLVTVSS,ARQGRFLDWLLYFEN,17
314,ACTATCTCAGGGAGAG-1.CM003-5-9,TGGGAGTGCTTTCTGAGAGTCATGGACCTCCTGCACAAGAACATGA...,F,T,IGHV4-34*01,IGHD3-9*01,IGHJ4*02,CAGGTGCAACTACAGCAGTGGGGCGCA...GGACTGTTGAAGCCTT...,CAGGTGCAGCTACAGCAGTGGGGCGCA...GGACTGTTGAAGCCTT...,TGTGCGAGACAGGGTTATGATATTTTGACTGGGGTGGAGCGACACT...,...,FDYWGQGTVVTVSS,FDYWGQGTLVTVSS,QVQLQQWGAGLLKPSETLSLTCAVN,GGSFSGYY,WTWIRQSPGKGLEWIGE,INHSGSIT,NYSPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYFC,WGQGTVVTVSS,ARQGYDILTGVERHFDY,19
659,CGTCAGGAGTCAATAG-1.CM003-1,GACCCTGCAGCTCTGGGAGAGGAGCCCAGCACTAGAAGTCGGCGGT...,F,T,IGHV3-33*01,"IGHD4-11*01,IGHD4-17*01,IGHD4-23*01",IGHJ6*02,CAGGTGCAGTTGCTGGAATCTGGGGGA...GGCGTGGTCCAGCCTG...,CAGGTGCAGCTGGTGGAGTCTGGGGGA...GGCGTGGTCCAGCCTG...,TGTGCGAGAGAGAGAGAGACTACCTTTTCCCACTACTACGGTATGG...,...,YYGMDVWGRGTTVTVSS,YYGMDVWGQGTTVTVSS,QVQLLESGGGVVQPGRSLRLSCAAS,GFTFSSYG,MHWVRQAPGKGLEWVTF,IWYDQNNK,YYADSVKGRFTISRDNSKKMLYLQMNSLRVEDTAVYYC,WGRGTTVTVSS,ARERETTFSHYYGMDV,18
